In [1]:
import requests
import time
session = requests.Session()
import pandas as pd
from bs4 import BeautifulSoup
import lxml
import xml.etree.ElementTree as ET
from tqdm import tqdm

In [5]:
### suburb and postal code will be a dictionary
HEADERS = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36",
            "accept": "application/json"}



In [120]:
def get_website(seed):
#     seed = f'https://www.domain.com.au/rent/{suburb}-vic-{postalcode}/?page={page_num}'
#     seed = f"https://www.domain.com.au/sitemap-listings-rent-2022080823.xml"
    response = session.get(seed, headers=HEADERS)
    data = response.text
    return data

### Get listing groups

In [130]:
listings_groups = []
try:
    data = get_website("https://www.domain.com.au/sitemap-listings-rent.xml")
    soup = BeautifulSoup(data, "html.parser")
    listings_groups = [x.contents[0] for x in soup.find_all('loc')]
    listings_groups = pd.DataFrame(listings_groups, columns=['links'])
    listings_groups.to_csv("../data/raw/listing_groups.csv")
except Exception as e:
    print(e)

/home/sousf/.local/lib/python3.10/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


### Get individual listings

In [142]:
listings_groups = pd.read_csv("../data/raw/listing_groups.csv")
listings_groups['links']
listings = []
try:
    for group in listings_groups['links']:
        data = get_website(group)
        soup = BeautifulSoup(data, "html.parser")
        listings.extend([x.contents[0] for x in soup.find_all('loc') if '-vic-' in x.contents[0]])
#         print(listings)
except Exception as e:
    print(e)
    
listings

/home/sousf/.local/lib/python3.10/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


['https://www.domain.com.au/1-sanctuary-way-ascot-vale-vic-3032-16065444',
 'https://www.domain.com.au/1206-639-little-lonsdale-street-melbourne-vic-3000-16065443',
 'https://www.domain.com.au/13-86-hotham-street-east-melbourne-vic-3002-16065438',
 'https://www.domain.com.au/60-777-whitehorse-road-mont-albert-vic-3127-16065435',
 'https://www.domain.com.au/15-vincent-avenue-st-albans-vic-3021-16065436',
 'https://www.domain.com.au/1-238-deakin-avenue-mildura-vic-3500-16065433',
 'https://www.domain.com.au/4-gila-way-truganina-vic-3029-16065421',
 'https://www.domain.com.au/unit-4-18-avoca-ave-elwood-vic-3184-16065420',
 'https://www.domain.com.au/6-percy-place-prahran-vic-3181-16047983',
 'https://www.domain.com.au/316-330-lygon-street-brunswick-east-vic-3057-15187572',
 'https://www.domain.com.au/61-dutton-lane-ascot-vale-vic-3032-16065419',
 'https://www.domain.com.au/1306-618-lonsdale-street-melbourne-vic-3000-16065416',
 'https://www.domain.com.au/38-fantail-way-brookfield-vic-3338

In [143]:
len(listings)

16025

In [145]:
listings = pd.DataFrame(listings, columns=['links'])
listings.to_csv("../data/raw/listings.csv", index=False)

### Get listing data

In [44]:
# response = session.get('https://www.domain.com.au/1-sanctuary-way-ascot-vale-vic-3032-16065444', headers=HEADERS)
response = session.get('https://www.domain.com.au/805-81-queens-road-melbourne-vic-3004-16067649', headers=HEADERS)

listing_data = response.json()

In [45]:
listing_data.keys()

dict_keys(['pageType', 'pageTitle', 'props', 'digitalData', 'raygunTags'])

In [46]:
listing_data['props']['id']

16067649

In [47]:
listing_data['props'].keys()

dict_keys(['user', 'loginAndReturnUrl', 'signupAndReturnUrl', 'notificationApiUrl', 'listingsMap', 'otherListingsIds', 'address', 'unitNumber', 'streetNumber', 'street', 'suburb', 'postcode', 'stateAbbreviation', 'projectColor', 'createdOn', 'modifiedOn', 'projectName', 'isStandardListing', 'isArchived', 'id', 'listingId', 'listingUrl', 'projectUrl', 'bigProjectLogo', 'smallProjectLogo', 'brandingProjectLogo', 'brandingProjectColor', 'footer', 'propertyType', 'beds', 'phone', 'agencyName', 'propertyDeveloperName', 'agencyLogo', 'agencyBanner', 'agencyProfileUrl', 'propertyDeveloperUrl', 'projectLink', 'websiteLink', 'brandingColor', 'brandingAppearance', 'limitedAgencyMode', 'listingType', 'description', 'fetchUrl', 'headline', 'tagline', 'banner', 'loanfinder', 'schoolCatchment', 'whatIsNearby', 'neighbourhoodInsights', 'locationProfileCards', 'suburbInsights', 'otherProjects', 'gallery', 'additionalLinks', 'header', 'listingSummary', 'propertyProfileUrlSlug', 'agents', 'inspection', 

In [48]:
listing_data['props']['listingUrl']

'https://www.domain.com.au/805-81-queens-road-melbourne-vic-3004-16067649'

In [67]:
stats = listing_data['props']['listingSummary']['stats']
stat_list = {}
for d in stats:
    stat = {}
    stat[d['key']] = d['value']
    stat_list.update(stat)
    
test_list = [stat_list, {'landsize': 432}]
pd.DataFrame(test_list)

,availableFrom,bond,landsize
0,"Friday, 2nd September 2022",1651.0,NaN
1,NaN,NaN,432.0


### Property data
price per week, postcode/suburb, address, bedrooms, bathrooms, carpark, property type, 

In [340]:
prop_data = listing_data['props']['listingSummary']
stat = prop_data.pop("stats", None)
prop_df = pd.DataFrame(prop_data, index=[0])
# prop_df['stampDutyEstimate'] = listing_data['props']['stampDutyEstimate']
prop_df['id'] = listing_data['props']['id']
prop_df

,beds,baths,parking,title,price,address,promoType,listingType,propertyType,status,mode,isRural,houses,showDefaultFeatures,id
0,2,1,1,$480 per week,$480 per week,"102/279 Wellington Parade South, East Melbourn...",branded,rent,Apartment / Unit / Flat,recentlyUpdated,rent,False,0,True,14787024


In [252]:
prop_data

[{'key': 'availableFrom',
  'value': 'Sunday, 4th September 2022',
  'iso8601Value': '2022-09-04T14:00:00'},
 {'key': 'bond', 'value': 2390}]

In [240]:
prop_data

{'beds': 4,
 'baths': 3,
 'parking': 2,
 'title': '$950',
 'price': '$950',
 'address': '1 Sanctuary Way, Ascot Vale VIC 3032',
 'promoType': 'platinum',
 'listingType': 'rent',
 'propertyType': 'House',
 'status': 'new',
 'mode': 'rent',
 'isRural': False,
 'houses': 0,
 'showDefaultFeatures': True}

### Suburb Data


In [267]:
listing_data['props']['suburbInsights']

{'beds': 1,
 'propertyType': 'Unit',
 'suburb': 'Melbourne',
 'suburbProfileUrl': '/suburb-profile/melbourne-vic-3000',
 'medianPrice': 330000,
 'medianRentPrice': 340,
 'avgDaysOnMarket': 121,
 'auctionClearance': 41,
 'nrSoldThisYear': 395,
 'entryLevelPrice': 180000,
 'luxuryLevelPrice': 593000,
 'renterPercentage': 0.6970133677881032,
 'singlePercentage': 0.7628455892048701,
 'demographics': {'population': 47279,
  'avgAge': '20 to 39',
  'owners': 0.3029866322118968,
  'renters': 0.6970133677881032,
  'families': 0.2371544107951299,
  'singles': 0.7628455892048701,
  'clarification': True},
 'salesGrowthList': [{'medianSoldPrice': 429000,
   'annualGrowth': 0,
   'numberSold': 665,
   'year': '2017'},
  {'medianSoldPrice': 360000,
   'annualGrowth': -0.16083916083916083,
   'numberSold': 468,
   'year': '2018'},
  {'medianSoldPrice': 355000,
   'annualGrowth': -0.013888888888888888,
   'numberSold': 487,
   'year': '2019'},
  {'medianSoldPrice': 361000,
   'annualGrowth': 0.016901

In [277]:
i = 0
# edit this to add or remove suburb features
suburb_keys = ["suburb", "medianRentPrice"]


suburb_data = {key: listing_data['props']['suburbInsights'][key] for key in suburb_keys}
suburb_data.update(listing_data['props']['suburbInsights']["demographics"])
suburb_data.pop("clarification")
suburb_data


suburb_df = pd.DataFrame(suburb_data, index = [i])
suburb_df

,suburb,medianRentPrice,population,avgAge,owners,renters,families,singles
0,Melbourne,340,47279,20 to 39,0.302987,0.697013,0.237154,0.762846


### Neighbourhood Data

In [281]:
listing_data['props']['neighbourhoodInsights']

{'age0To19': 0.05878031,
 'age20To39': 0.775900066,
 'age40To59': 0.138868481,
 'age60Plus': 0.0264511388,
 'longTermResident': 0.120689653,
 'owner': 0.199685544,
 'renter': 0.8003145,
 'family': 0.3400703,
 'single': 0.6599297,
 'showIncomeAndExpenses': False}

In [288]:
i = 0
neighbourhood_data = listing_data['props']['neighbourhoodInsights']
neighbourhood_data.pop('map')
neighbourhood_data.pop('showIncomeAndExpenses')
neighbourhood_df = pd.DataFrame(neighbourhood_data, index = [i])
neighbourhood_df

,age0To19,age20To39,age40To59,age60Plus,longTermResident,owner,renter,family,single
0,0.05878,0.7759,0.138868,0.026451,0.12069,0.199686,0.800315,0.34007,0.65993


In [289]:
neighbourhood_df

,age0To19,age20To39,age40To59,age60Plus,longTermResident,owner,renter,family,single
0,0.05878,0.7759,0.138868,0.026451,0.12069,0.199686,0.800315,0.34007,0.65993


### Nearest schools

In [292]:
closest_school = listing_data['props']['schoolCatchment']['schools']
closest_school

[{'id': '',
  'educationLevel': 'secondary',
  'name': 'Ozford College - Ozford College Campus',
  'distance': 259.55083543938684,
  'state': 'VIC',
  'postCode': '3000',
  'year': '',
  'gender': '',
  'type': 'Private',
  'address': 'Melbourne, VIC 3000',
  'url': '',
  'domainSeoUrlSlug': 'ozford-college-english-language-centre-vic-3000-52094'},
 {'id': '',
  'educationLevel': 'secondary',
  'name': 'Ozford College',
  'distance': 260.5021570858975,
  'state': 'VIC',
  'postCode': '3000',
  'year': '11-12',
  'gender': 'CoEd',
  'type': 'Private',
  'address': 'Melbourne, VIC 3000',
  'url': 'http://www.ozford.edu.au',
  'domainSeoUrlSlug': 'ozford-college-vic-3000-40714'},
 {'id': '',
  'educationLevel': 'combined',
  'name': 'Haileybury Girls College - City Campus',
  'distance': 351.66409019499406,
  'state': 'VIC',
  'postCode': '3006',
  'year': '',
  'gender': '',
  'type': 'Private',
  'address': 'West Melbourne, VIC 3006',
  'url': '',
  'domainSeoUrlSlug': 'haileybury-girls

In [331]:
closest_schools = listing_data['props']['schoolCatchment']['schools']
closest_schools

[{'id': '',
  'educationLevel': 'secondary',
  'name': 'SEDA College (Victoria) - Netball Ascot Vale',
  'distance': 407.0349574515122,
  'state': 'VIC',
  'postCode': '3032',
  'year': '',
  'gender': '',
  'type': 'Private',
  'address': 'Ascot Vale, VIC 3032',
  'url': '',
  'domainSeoUrlSlug': 'seda-college-victoria-netball-ascot-vale-vic-3032-52561'},
 {'id': '10491',
  'educationLevel': 'primary',
  'name': "St Mary's School",
  'distance': 726.0180056952819,
  'state': 'VIC',
  'postCode': '3032',
  'year': 'Prep-6',
  'gender': 'CoEd',
  'type': 'Catholic',
  'address': 'Ascot Vale, VIC 3032',
  'url': 'http://www.smascotvale.catholic.edu.au/',
  'domainSeoUrlSlug': 'st-marys-school-vic-3032-10491'},
 {'id': '940',
  'educationLevel': 'primary',
  'name': 'Ascot Vale West Primary School',
  'distance': 816.3605116347898,
  'state': 'VIC',
  'postCode': '3032',
  'year': 'Prep-6',
  'gender': 'CoEd',
  'type': 'Government',
  'address': 'Ascot Vale, VIC 3032',
  'url': 'http://w

In [310]:
i = 0
top = closest_schools[0]
top = pd.DataFrame(top, index = [i])
top

,id,educationLevel,name,distance,state,postCode,year,gender,type,address,url,domainSeoUrlSlug
0,,secondary,Ozford College - Ozford College Campus,259.550835,VIC,3000,,,Private,"Melbourne, VIC 3000",,ozford-college-english-language-centre-vic-300...


In [3]:
listings = pd.read_csv("../data/raw/listings.csv")
listings

,links
0,https://www.domain.com.au/1-sanctuary-way-asco...
1,https://www.domain.com.au/1206-639-little-lons...
2,https://www.domain.com.au/13-86-hotham-street-...
3,https://www.domain.com.au/60-777-whitehorse-ro...
4,https://www.domain.com.au/15-vincent-avenue-st...
...,...
16020,https://www.domain.com.au/4-181-great-ocean-ro...
16021,https://www.domain.com.au/53-mcewan-road-heide...
16022,https://www.domain.com.au/14-8-clyde-street-es...
16023,https://www.domain.com.au/52-121-rathdowne-st-...


In [30]:
prop_list = []
sub_list = []
neigh_list = []
school_list = []
for i in tqdm(range(0, len(listings))):
    try:
        response = session.get(listings['links'][i], headers=HEADERS)
        listing_data = response.json()
        
        if ('redirect' in listing_data.keys()):
            print("Redirected: ", i, listings['links'][i])
            response = session.get("https://www.domain.com.au/" + listing_data['redirect']['destination'], headers=HEADERS)
            listing_data = response.json()
        elif ('props' not in listing_data.keys()):
            print("No props at index: ", i, listings['links'][i])
            
        

        # create property dataset
        prop_data = listing_data['props']['listingSummary']
        prop_data.pop("stats", None)
        prop_data.pop("showDefaultFeatures", None)
        prop_data.pop("tag", None)
        prop_data['id'] = listing_data['props']['id']
        prop_list.append(prop_data)


        # create suburb dataset
        # edit this to add or remove suburb features
        suburb_keys = ["suburb", "medianRentPrice"]


        suburb_data = {key: listing_data['props']['suburbInsights'][key] for key in suburb_keys}
        suburb_data.update(listing_data['props']['suburbInsights']["demographics"])
        suburb_data.pop("clarification", None)
        suburb_data['id'] = listing_data['props']['id']
        sub_list.append(suburb_data)
    #     suburb_df = pd.DataFrame(suburb_data, index = [i])
    #     suburb_df


        # create neighbourhood dataset
        neighbourhood_data = listing_data['props']['neighbourhoodInsights']
        neighbourhood_data.pop('map', None)
        neighbourhood_data.pop('showIncomeAndExpenses', None)
        neighbourhood_data['id'] = listing_data['props']['id']
        neigh_list.append(neighbourhood_data)
    #     neighbourhood_df = pd.DataFrame(neighbourhood_data, index = [i])
    #     neighbourhood_df


        # create closest school dataset
        closest_schools = listing_data['props']['schoolCatchment']['schools']
        top = closest_schools[0]
        top['id'] = listing_data['props']['id']
        top.pop("domainSeoUrlSlug", None)
        top.pop("isRadiusResult", None)
        school_list.append(top)
        time.sleep(0.3)
    except Exception as e:
        print(e)
        
    
prop_list = pd.DataFrame(prop_list)
sub_list = pd.DataFrame(sub_list)
neigh_list = pd.DataFrame(neigh_list)
school_list = pd.DataFrame(school_list)

  0%|▏                                                                             | 39/16025 [00:18<2:07:02,  2.10it/s]

Redirected:  39 https://www.domain.com.au/15-1-hatfeild-ct-west-footscray-vic-3012-16065321


  1%|▌                                                                            | 113/16025 [00:55<2:10:24,  2.03it/s]

Redirected:  113 https://www.domain.com.au/1506b-250-spencer-street-melbourne-vic-3000-14620499


  1%|▊                                                                            | 159/16025 [01:23<2:41:42,  1.64it/s]

...

100%|██████████████████████████████████████████████████████████████████████████▉| 16022/16025 [2:25:52<00:01,  2.03it/s]

Redirected:  16022 https://www.domain.com.au/14-8-clyde-street-essendon-vic-3040-14529998


100%|██████████████████████████████████████████████████████████████████████████▉| 16024/16025 [2:25:54<00:00,  1.80it/s]

Redirected:  16024 https://www.domain.com.au/2-19-eastgate-street-pascoe-vale-vic-3044-15376920


100%|███████████████████████████████████████████████████████████████████████████| 16025/16025 [2:25:55<00:00,  1.83it/s]


In [28]:

response = session.get("https://www.domain.com.au/14-8-clyde-street-essendon-vic-3040-14529998", headers=HEADERS)
listing_data = response.json()

In [ ]:
https://www.domain.com.au/413-40-hall-st-essendon-vic-3040-14529998

In [29]:
listing_data

{'redirect': {'destination': '413-40-hall-st-essendon-vic-3040-14529998',
  'statusCode': 301}}

In [31]:
prop_list

,beds,baths,parking,title,price,address,promoType,listingType,propertyType,status,mode,isRural,houses,id,method
0,4,3,2,$950,$950,"1 Sanctuary Way, Ascot Vale VIC 3032",platinum,rent,House,new,rent,False,0,16065444,NaN
1,1,1,0,$550 per week,$550 per week,"1206/639 Little Lonsdale Street, Melbourne VIC...",branded,rent,Apartment / Unit / Flat,new,rent,False,0,16065443,NaN
2,1,1,1,$475 per week,$475 per week,"13/86 Hotham Street, East Melbourne VIC 3002",platinum,rent,Apartment / Unit / Flat,new,rent,False,0,16065438,NaN
3,2,1,1,$450 per week,$450 per week,"60/777 Whitehorse Road, Mont Albert VIC 3127",branded,rent,Apartment / Unit / Flat,new,rent,False,0,16065435,NaN
4,3,1,4,$370 per week,$370 per week,"15 Vincent Avenue, St Albans VIC 3021",basic,rent,House,new,rent,False,0,16065436,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15535,3,2,2,from $400 per night off peak,from $400 per night off peak,"4/181 Great Ocean Road, Apollo Bay VIC 3233",basic,rent,Townhouse,live,rent,False,0,13210984,NaN
15536,0,1,0,$200 including bills pw,$200 including bills pw,"53 McEwan Road, Heidelberg Heights VIC 3081",branded,rent,Apartment / Unit / Flat,live,rent,False,0,15381771,NaN
15537,1,1,1,$450,$450,"413/40 Hall St, Essendon VIC 3040",basic,rent,Apartment / Unit / Flat,live,rent,False,0,14529998,NaN
15538,3,1,1,$780 weekly,$780 weekly,"52/121 Rathdowne St, Carlton VIC 3053",branded,rent,Apartment / Unit / Flat,live,rent,False,0,15378664,NaN


In [32]:
sub_list

,suburb,medianRentPrice,population,avgAge,owners,renters,families,singles,id
0,Ascot Vale,790,14758,20 to 39,0.573788,0.426212,0.418693,0.581307,16065444
1,Melbourne,340,47279,20 to 39,0.302987,0.697013,0.237154,0.762846,16065443
2,East Melbourne,395,4963,20 to 39,0.445647,0.554353,0.361296,0.638704,16065438
3,Mont Albert,420,4847,40 to 59,0.717633,0.282367,0.526342,0.473658,16065435
4,St Albans,360,37322,20 to 39,0.639986,0.360014,0.461730,0.538270,16065436
...,...,...,...,...,...,...,...,...,...
15511,Apollo Bay,0,1594,60+,0.660473,0.339527,0.462044,0.537956,13210984
15512,Heidelberg Heights,0,6063,20 to 39,0.592479,0.407521,0.380590,0.619410,15381771
15513,Essendon,300,20590,20 to 39,0.621521,0.378479,0.478467,0.521533,14529998
15514,Carlton,625,18536,20 to 39,0.194165,0.805835,0.149739,0.850261,15378664


In [33]:
neigh_list

,age0To19,age20To39,age40To59,age60Plus,longTermResident,owner,renter,family,single,id
0,0.291534,0.281150,0.313099,0.114217,0.398422,0.698876,0.301124,0.672790,0.327210,16065444
1,0.058780,0.775900,0.138868,0.026451,0.120690,0.199686,0.800315,0.340070,0.659930,16065443
2,0.156915,0.345745,0.252660,0.244681,0.477941,0.515337,0.484663,0.570946,0.429054,16065438
3,0.284697,0.261566,0.286477,0.167260,0.447316,0.619718,0.380282,0.606796,0.393204,16065435
4,0.204176,0.371230,0.229698,0.194896,0.534031,0.554622,0.445378,0.433022,0.566978,16065436
...,...,...,...,...,...,...,...,...,...,...
15511,0.185321,0.188991,0.260550,0.365138,0.587601,0.540909,0.459091,0.530120,0.469880,13210984
15512,0.200608,0.386018,0.255319,0.158055,0.649254,0.614173,0.385827,0.446154,0.553846,15381771
15513,0.167059,0.357647,0.263529,0.211765,0.443299,0.535912,0.464088,0.497041,0.502959,14529998
15514,0.078680,0.573604,0.208122,0.139594,0.201835,0.335404,0.664596,0.462500,0.537500,15378664


In [34]:
school_list

,id,educationLevel,name,distance,state,postCode,year,gender,type,address,url
0,16065444,secondary,SEDA College (Victoria) - Netball Ascot Vale,407.034957,VIC,3032,,,Private,"Ascot Vale, VIC 3032",
1,16065443,secondary,Ozford College - Ozford College Campus,259.550835,VIC,3000,,,Private,"Melbourne, VIC 3000",
2,16065438,primary,Richmond West Primary School,729.249072,VIC,3121,Prep-6,CoEd,Government,"Richmond, VIC 3121",http://www.richmondwestps.vic.edu.au
3,16065435,primary,Mont Albert Primary School,671.793634,VIC,3127,Prep-6,CoEd,Government,"Mont Albert, VIC 3127",http://www.maps.vic.edu.au
4,16065436,primary,St Albans East Primary School,206.943539,VIC,3021,Prep-6,CoEd,Government,"St Albans, VIC 3021",http://www.saeps.vic.edu.au
...,...,...,...,...,...,...,...,...,...,...,...
15509,13210984,combined,Apollo Bay P-12 College,720.992925,VIC,3233,Prep-12,CoEd,Government,"Apollo Bay, VIC 3233",http://external.apollobayp12.vic.edu.au/
15510,15381771,primary,Rosanna Primary School,898.177717,VIC,3084,Prep-6,CoEd,Government,"Rosanna, VIC 3084",http://www.rosannaps.vic.edu.au
15511,14529998,primary,St Monica's Primary School,705.284820,VIC,3039,Prep-6,CoEd,Catholic,"Moonee Ponds, VIC 3039",http://www.stmonicasmp.catholic.edu.au
15512,15378664,primary,Carlton Gardens Primary School,244.869231,VIC,3053,Prep-6,CoEd,Government,"Carlton, VIC 3053",http://www.carltongardens.vic.edu.au


In [35]:
prop_list.to_csv('../data/raw/listing_properties.csv')
sub_list.to_csv('../data/raw/suburb_stat.csv')
neigh_list.to_csv('../data/raw/neigh_stat.csv')
school_list.to_csv('../data/raw/closest_school.csv')

In [222]:
listing_data['props']['locationProfileCards']

{'profiles': [{'type': 'street',
   'profileUrl': 'https://www.domain.com.au/street-profile/sanctuary-walk-ascot-vale-vic-3032',
   'onMarketType': 'rent',
   'address': {'streetName': 'Sanctuary',
    'streetType': 'Wk',
    'streetTypeLong': 'Walk',
    'suburb': 'Ascot Vale',
    'postcode': '3032',
    'state': 'VIC'},
   'statistics': {'owners': 7,
    'tenants': 4,
    'propertyTypes': [{'type': 'Unit',
      'total': 2,
      'forSale': 0,
      'forRent': 0,
      'soldLast12Months': 0,
      'rentedLast12Months': 0},
     {'type': 'Townhouse',
      'total': 1,
      'forSale': 0,
      'forRent': 0,
      'soldLast12Months': 0,
      'rentedLast12Months': 0},
     {'type': 'House',
      'total': 9,
      'forSale': 0,
      'forRent': 1,
      'soldLast12Months': 1,
      'rentedLast12Months': 1},
     {'type': 'Other',
      'total': 5,
      'forSale': 0,
      'forRent': 0,
      'soldLast12Months': 0,
      'rentedLast12Months': 0}],
    'averageOwnerOccupierTenureMonths